In [2]:
# تثبيت mediapipe بشكل نظيف
!pip install --upgrade pip setuptools wheel
!pip install --upgrade protobuf
!pip install mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.30.2 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.30.2 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.30.2 which is incompatible.
google-cloud-aiplatform 1.87.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0,>=3.20.2, but you have protobuf 6.30.2 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.2

In [20]:
#  تثبيت المكتبات المطلوبة
!pip install mediapipe ultralytics opencv-python numpy pandas openpyxl

#  ربط Google Drive
from google.colab import drive
drive.mount('/content/drive')

#  استيراد المكتبات
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
import pandas as pd
from google.colab import files
import os

#  إدخال اسم اللاعب ورفع الفيديو
player_name = input(" أدخلي اسم اللاعب: ")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

#  تحميل نموذج YOLO وتتبع الجسم
model = YOLO("yolov8n.pt")
pose = mp.solutions.pose.Pose()

#  فتح الفيديو وتحضيره
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("scout_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

#  تعريف المتغيرات
touch_count = 0
min_touch_distance = 35
stability_scores = []
skill_score = 0
ball_positions = []
direction_changes = 0
show_touch_effect = 0

#  دوال المساعدة
def get_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_angle_change(a, b):
    return abs(np.arctan2(b[1]-a[1], b[0]-a[0]))

#  تحليل كل إطار
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)
    pose_result = pose.process(rgb)

    ball_center = None
    player_foot = None
    touched = False

    # تحديد موقع الكرة
    for det in results[0].boxes:
        x1, y1, x2, y2 = map(int, det.xyxy[0])
        cls = int(det.cls[0])
        if cls == 32:
            ball_center = ((x1 + x2)//2, (y1 + y2)//2)
            cv2.circle(frame, ball_center, 6, (0, 255, 255), -1)

    # تتبع قدم اللاعب وتقدير الثبات
    if pose_result.pose_landmarks:
        lm = pose_result.pose_landmarks.landmark
        foot = lm[mp.solutions.pose.PoseLandmark.RIGHT_FOOT_INDEX]
        hip = lm[mp.solutions.pose.PoseLandmark.RIGHT_HIP]
        fx, fy = int(foot.x * w), int(foot.y * h)
        hx, hy = int(hip.x * w), int(hip.y * h)
        player_foot = (fx, fy)
        movement = abs(hip.y - foot.y)
        stability = max(0, 1 - movement * 3)
        stability_scores.append(stability)
        cv2.circle(frame, player_foot, 5, (0, 0, 255), -1)

    # حساب اللمسة
    if ball_center and player_foot:
        dist = get_distance(ball_center, player_foot)
        if dist < min_touch_distance:
            touch_count += 1
            touched = True
            show_touch_effect = 5

    if touched and ball_center:
        cv2.circle(frame, ball_center, 20, (0, 255, 0), 3)
    elif show_touch_effect > 0 and ball_center:
        cv2.circle(frame, ball_center, 20, (0, 255, 0), 2)
        show_touch_effect -= 1

    # حساب تغييرات الاتجاه
    if ball_center:
        ball_positions.append(ball_center)
        if len(ball_positions) >= 6:
            a, b = ball_positions[-6], ball_positions[-1]
            angle = get_angle_change(a, b)
            if 'last_angle' in locals() and abs(angle - last_angle) > 0.6:
                direction_changes += 1
                skill_score += 1
            last_angle = angle

    avg_stability = sum(stability_scores) / len(stability_scores) if stability_scores else 0

    #  عرض النتائج على الفيديو
    cv2.putText(frame, f"Touches: {touch_count}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
    cv2.putText(frame, f"Stability: {avg_stability:.2f}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    cv2.putText(frame, f"Skill: {skill_score}", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,128,255), 2)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

#  تصنيف مستوى اللاعب
level = "متوسط"
if touch_count >= 12 and skill_score >= 4 and avg_stability > 0.7:
    level = "احترافي"
elif touch_count >= 8 and avg_stability > 0.5:
    level = "مستوى عالي"

#  حفظ النتائج في Excel داخل Google Drive
excel_path = "/content/drive/MyDrive/ScoutPlayResults/sorted_players_updated.xlsx"

# إضافة النتائج في الصف الأخير بالأعمدة المخصصة
df = pd.read_excel(excel_path)
new_row = {
    "Name": player_name,
    "Touches": touch_count,
    "Skill Score": skill_score,
    "Avg Stability": round(avg_stability, 2),
    "Level": level,
    "Video": video_path
}
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
df.to_excel(excel_path, index=False)

print(" تم حفظ النتائج في الملف بنجاح!")

#  تحميل الفيديو النهائي
files.download("scout_output.mp4")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 أدخلي اسم اللاعب: احمد


Saving ogTIkffnnAXjDlDIgh7EA8DIeL5FQhgqPChStn.mp4 to ogTIkffnnAXjDlDIgh7EA8DIeL5FQhgqPChStn.mp4

0: 640x384 1 person, 1 sports ball, 106.3ms
Speed: 2.9ms preprocess, 106.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 90.1ms
Speed: 3.2ms preprocess, 90.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 96.4ms
Speed: 2.4ms preprocess, 96.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 96.5ms
Speed: 3.2ms preprocess, 96.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 91.2ms
Speed: 2.9ms preprocess, 91.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 90.2ms
Speed: 2.8ms preprocess, 90.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 sports ball, 91.3ms
Speed: 2.7ms

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>